In [1]:
import pandas as pd

In [2]:
train_demand_data_2018 = pd.read_csv('../data/raw/Train_Service_Passenger_Counts_Financial_Year_2018-2019.csv')
train_demand_data_2019 = pd.read_csv('../data/raw/Train_Service_Passenger_Counts_Financial_Year_2019-2020.csv')
train_demand_data_2022 = pd.read_csv('../data/raw/Train_Service_Passenger_Counts_Financial_Year_2021-2022.csv')
train_demand_data_2023 = pd.read_csv('../data/raw/Train_Service_Passenger_Counts_Financial_Year_2022-2023.csv')

train_demand_data_precovid = pd.concat([train_demand_data_2018, train_demand_data_2019])
train_demand_data_postcovid = pd.concat([train_demand_data_2022, train_demand_data_2023])

In [ ]:
def clean(train_demand_data, analysis_type):
    # clean out vline
    train_demand_data = train_demand_data[train_demand_data['Mode'] != 'VLine']

    train_demand_data['year'] = pd.to_datetime(train_demand_data['Business_Date']).dt.year

    if analysis_type == 'precovid':
        train_demand_data = train_demand_data[train_demand_data['year'] < 2020]
    elif analysis_type == 'postcovid':
        train_demand_data = train_demand_data[train_demand_data['year'] > 2021]

    # get_datetime
    train_demand_data['Departure_3hr_Time_Bracket'] = train_demand_data['Departure_Time_Scheduled'].apply(lambda x: 0 if '00:00:00' <= x < '03:00:00' else \
                                                                                                            1 if '03:00:00' <= x < '06:00:00' else \
                                                                                                                2 if '06:00:00' <= x < '09:00:00' else \
                                                                                                                    3 if '09:00:00' <= x < '12:00:00' else \
                                                                                                                        4 if '12:00:00' <= x < '15:00:00' else \
                                                                                                                            5 if '15:00:00' <= x < '18:00:00' else \
                                                                                                                                6 if '18:00:00' <= x < '21:00:00' else \
                                                                                                                                    7)
    train_demand_data['Arrival_3hr_Time_Bracket'] = train_demand_data['Arrival_Time_Scheduled'].apply(lambda x: 0 if '00:00:00' <= x < '03:00:00' else \
                                                                                                            1 if '03:00:00' <= x < '06:00:00' else \
                                                                                                                2 if '06:00:00' <= x < '09:00:00' else \
                                                                                                                    3 if '09:00:00' <= x < '12:00:00' else \
                                                                                                                        4 if '12:00:00' <= x < '15:00:00' else \
                                                                                                                            5 if '15:00:00' <= x < '18:00:00' else \
                                                                                                                                6 if '18:00:00' <= x < '21:00:00' else \
                                                                                                                                    7)
    
    return train_demand_data

In [ ]:
def get_mean_daily_demand(train_demand_data):
    
    daily_deamnd = train_demand_data[['Station_Name', 'Business_Date', 'Passenger_Boardings', 'Passenger_Alightings']].groupby(['Station_Name', 'Business_Date']).agg('sum')

    mean_daily_demand = daily_deamnd.groupby('Station_Name').agg('mean').sort_values(by='Passenger_Boardings', ascending=False)

    return mean_daily_demand

In [ ]:
train_demand_data_pre_covid = clean(train_demand_data_precovid, 'precovid')
mean_daily_demand_pre_covid = get_mean_daily_demand(train_demand_data_pre_covid)
mean_daily_demand_pre_covid['Total_Demand'] = mean_daily_demand_pre_covid['Passenger_Boardings'] + mean_daily_demand_pre_covid['Passenger_Alightings']
mean_daily_demand_pre_covid.to_csv('../data/curated/mean_daily_trains_demand_pre_covid.csv')

train_demand_data_post_covid = clean(train_demand_data_postcovid, 'postcovid')
mean_daily_demand_post_covid = get_mean_daily_demand(train_demand_data_post_covid)
mean_daily_demand_post_covid['Total_Demand'] = mean_daily_demand_post_covid['Passenger_Boardings'] + mean_daily_demand_post_covid['Passenger_Alightings']
mean_daily_demand_post_covid.to_csv('../data/curated/mean_daily_trains_demand_post_covid.csv')

/var/folders/tm/j_z94_gs1rncc1vngr4gz39m0000gn/T/ipykernel_3485/834488481.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_demand_data['Departure_3hr_Time_Bracket'] = train_demand_data['Departure_Time_Scheduled'].apply(lambda x: 0 if '00:00:00' <= x < '03:00:00' else \
/var/folders/tm/j_z94_gs1rncc1vngr4gz39m0000gn/T/ipykernel_3485/834488481.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_demand_data['Arrival_3hr_Time_Bracket'] = train_demand_data['Arrival_Time_Scheduled'].apply(lambda 

- get station and line information

In [ ]:
import geopandas as gpd

In [ ]:
train_stops = gpd.read_file('../data/raw/Metro Stops/PTV/PTV_METRO_TRAIN_STATION.shp')

In [ ]:
train_stops = gpd.read_file('../data/raw/Metro Stops/EXTRACT_POLYGON.shp')

In [ ]:
train_stops

,d_y_n_ID,geometry
0,None,"POLYGON ((280988.665 5798001.482, 280988.670 5..."
